In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.transforms import v2


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as Datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.models as models
import torchvision.utils as vutils
from torch.hub import load_state_dict_from_url
from torchvision import datasets, transforms
from torch.utils.data import random_split

import os
import random
import numpy as np
import math
from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import trange, tqdm
from torchvision.transforms import functional as TF
import PIL

batch_size = 64
image_size = 128
lr = 1e-4
nepoch = 100
start_epoch = 0


dataset_root = ""
save_dir = os.getcwd()
model_name = "Wasserstein_AE_oxford"
load_checkpoint  = True


use_cuda = torch.cuda.is_available()
gpu_indx  = 0
device = torch.device(gpu_indx if use_cuda else "cpu")

In [ ]:
import torch
from PIL import Image
from torch.utils.data import Dataset
import os

class CustomImageDataset(Dataset):
    def __init__(self, data_dir, file_names, transform=None, num_black_images=350, num_white_images=350, image_size=(3, 128, 128) ):
        self.data_dir = data_dir
        self.transform = transform
        self.file_names = file_names
        self.image_size = image_size

        # Adad placeholders for black images
        self.black_image_placeholder = "<black_image>"
        self.file_names.extend([self.black_image_placeholder] * num_black_images)
        self.white_image_placeholder = "<white_image>"
        self.file_names.extend([self.white_image_placeholder] * num_white_images)

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        if self.file_names[idx] == self.black_image_placeholder:
            # Create a black image
            black_image = torch.zeros(self.image_size)
            black_image = TF.to_pil_image(black_image)
            return transforms.ToTensor()(black_image) #self.transform(black_image) if self.transform else black_image

        if self.file_names[idx] == self.white_image_placeholder:
            # Create a black image
            white_image = torch.ones(self.image_size)
            white_image = TF.to_pil_image(white_image)
            return transforms.ToTensor()(white_image) #self.transform(white_image) if self.transform else black_image


        img_name = os.path.join(self.data_dir, self.file_names[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

train_transform = transforms.Compose([
    transforms.Resize(size=(image_size, image_size)),
    v2.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([transforms.Resize(size=(image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
#!pip install scikit-learn

from sklearn.model_selection import train_test_split

# Assuming all your images are in 'data_dir'
all_images = [img for img in os.listdir(dataset_root) if img.endswith('.jpg')]  # Adjust for your file type
train_images, test_images = train_test_split(all_images, test_size=0.05, random_state=7)

train_dataset = CustomImageDataset(data_dir=dataset_root, file_names=train_images, transform=train_transform, num_black_images=500)
test_dataset = CustomImageDataset(data_dir=dataset_root, file_names=test_images, transform=test_transform, num_black_images=500)




In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
dataiter = iter(train_loader)
train_images = next(dataiter)
train_images.shape

test_dataiter = iter(test_loader)
test_images = next(test_dataiter)
test_images.shape

In [ ]:
plt.figure(figsize = (20,10))
out = vutils.make_grid(test_images[0:64], normalize=True)
plt.imshow(out.numpy().transpose((1, 2, 0)))

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from typing import List, Callable, Union, Any, TypeVar, Tuple
from dataclasses import field

Tensor = TypeVar('torch.tensor')
class ResDown(nn.Module):
    """
    Residual down sampling block for the encoder
    """

    def __init__(self, channel_in, channel_out, kernel_size=3):
        super(ResDown, self).__init__()
        self.conv1 = nn.Conv2d(channel_in, channel_out // 2, kernel_size, 2, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_out // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_out // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 2, kernel_size // 2)

        self.act_fnc = nn.ELU()

    def forward(self, x):
        skip = self.conv3(x)
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)
        return self.act_fnc(self.bn2(x + skip))


class ResUp(nn.Module):
    """
    Residual up sampling block for the decoder
    """

    def __init__(self, channel_in, channel_out, kernel_size=3, scale_factor=2):
        super(ResUp, self).__init__()

        self.conv1 = nn.Conv2d(channel_in, channel_in // 2, kernel_size, 1, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_in // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_in // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 1, kernel_size // 2)

        self.up_nn = nn.Upsample(scale_factor=scale_factor, mode="nearest")

        self.act_fnc = nn.ELU()

    def forward(self, x):
        x = self.up_nn(x)
        skip = self.conv3(x)
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)

        return self.act_fnc(self.bn2(x + skip))


class ResBlock(nn.Module):
    """
    Residual block
    """

    def __init__(self, channel_in, channel_out, kernel_size=3):
        super(ResBlock, self).__init__()

        self.conv1 = nn.Conv2d(channel_in, channel_in // 2, kernel_size, 1, kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(channel_in // 2, eps=1e-4)
        self.conv2 = nn.Conv2d(channel_in // 2, channel_out, kernel_size, 1, kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(channel_out, eps=1e-4)

        if not channel_in == channel_out:
            self.conv3 = nn.Conv2d(channel_in, channel_out, kernel_size, 1, kernel_size // 2)
        else:
            self.conv3 = nn.Identity()

        self.act_fnc = nn.ELU()

    def forward(self, x):
        skip = self.conv3(x)
        x = self.act_fnc(self.bn1(self.conv1(x)))
        x = self.conv2(x)

        return self.act_fnc(self.bn2(x + skip))


class Encoder(nn.Module):
    """
    Encoder block
    """

    def __init__(self, channels, ch=64, blocks=(1, 2, 4, 8), latent_channels=512):
        super(Encoder, self).__init__()
        self.conv_in = nn.Conv2d(channels, blocks[0] * ch, 3, 1, 1)

        widths_in = list(blocks)
        widths_out = list(blocks[1:]) + [blocks[-1]]

        layer_blocks = []

        for w_in, w_out in zip(widths_in, widths_out):
            layer_blocks.append(ResDown(w_in * ch, w_out * ch))

        layer_blocks.append(ResBlock(blocks[-1] * ch, blocks[-1] * ch))
        layer_blocks.append(ResBlock(blocks[-1] * ch, blocks[-1] * ch))

        self.res_blocks = nn.Sequential(*layer_blocks)

        self.conv_mu = nn.Conv2d(blocks[-1] * ch, latent_channels, 1, 1)
        self.act_fnc = nn.ELU()



    def forward(self, x, sample=False):
        x = self.act_fnc(self.conv_in(x))
        x = self.res_blocks(x)
        z = self.conv_mu(x)
        return z


class Decoder(nn.Module):
    """
    Decoder block
    Built to be a mirror of the encoder block
    """

    def __init__(self, channels, ch=64, blocks=(1, 2, 4, 8), latent_channels=512):
        super(Decoder, self).__init__()
        self.conv_in = nn.Conv2d(latent_channels, ch * blocks[-1], 1, 1)

        widths_out = list(blocks)[::-1]
        widths_in = (list(blocks[1:]) + [blocks[-1]])[::-1]

        layer_blocks = [ResBlock(blocks[-1] * ch, blocks[-1] * ch),
                        ResBlock(blocks[-1] * ch, blocks[-1] * ch)]

        for w_in, w_out in zip(widths_in, widths_out):
            layer_blocks.append(ResUp(w_in * ch, w_out * ch))

        self.res_blocks = nn.Sequential(*layer_blocks)

        self.conv_out = nn.Conv2d(blocks[0] * ch, channels, 3, 1, 1)
        self.act_fnc = nn.ELU()

    def forward(self, x):
        x = self.act_fnc(self.conv_in(x))
        x = self.res_blocks(x)
        mu = torch.tanh(self.conv_out(x))
        return mu

class WassersteinAE(nn.Module):

    def __init__(self,
                 channel_in: int = 3,
                 ch=64,
                 blocks=(1, 2, 4, 8),
                 latent_channels=512,
                 latent_dim = 4096,
                 kernel_bandwidth = 1.0,
                 kernel_choice = "imq",
                 scales = [0.1, 0.2, 0.5, 1.0, 2.0, 5, 10.0],
                 reconstruction_loss_scale = 1.0,
                 reg_weight = 3e-2
                 ):
        super(WassersteinAE, self).__init__()

        self.latent_dim = latent_dim
        self.kernel_bandwidth = kernel_bandwidth

        self.kernel_choice = kernel_choice
        self.scales = scales #model_config.scales if model_config.scales is not None else [1.0]
        self.reconstruction_loss_scale = reconstruction_loss_scale #self.model_config.reconstruction_loss_scale
        self.reg_weight = reg_weight


        self.encoder = Encoder(channel_in, ch=ch, blocks=blocks, latent_channels=latent_channels)
        self.decoder = Decoder(channel_in, ch=ch, blocks=blocks, latent_channels=latent_channels)



    def encode(self, input: Tensor) -> Tensor:
        result = self.encoder(input)
        #print("After encoder:", result.shape)
        #result = torch.flatten(result, start_dim=1)
        #print("After flattening:", result.shape)
        #encoded = self.fc_encode(result)
        #print("After linear layer:", encoded.shape)
        return result

    def decode(self, z: Tensor) -> Tensor:
        #result = self.decoder_input(z)
        #print("After linear (decoder input):", result.shape)

        #result = result.view(-1, 512, 2, 2)
        #print("After reshaping:", result.shape)

        result = self.decoder(z)
        #print("After decoder:", result.shape)
        #result = self.final_upsample(result)

        # Apply the final convolutional layer
        #result = self.final_layer(result)

        #print("After final layer:", result.shape)


        return result

    def forward(self, x: Tensor) -> Tensor:
        z = self.encode(x)
        recon_x = self.decode(z)
        z_prior = torch.randn_like(z, device=x.device)
        loss, recon_loss, mmd_loss = self.loss_function(recon_x, x, z, z_prior)

        return {
            'recon_loss': recon_loss,
            'mmd_loss': mmd_loss,
            'loss': loss,
            'recon_x': recon_x,
            'z': z}


    def loss_function(self, recon_x, x, z, z_prior):
        N = z.shape[0]  # batch size

        recon_loss = self.reconstruction_loss_scale * F.mse_loss(
            recon_x.reshape(x.shape[0], -1), x.reshape(x.shape[0], -1), reduction="none"
        ).sum(dim=-1)


        if self.kernel_choice == "rbf":
            k_z = self.rbf_kernel(z, z)
            k_z_prior = self.rbf_kernel(z_prior, z_prior)
            k_cross = self.rbf_kernel(z, z_prior)

        else:
            k_z = self.imq_kernel(z, z)
            k_z_prior = self.imq_kernel(z_prior, z_prior)
            k_cross = self.imq_kernel(z, z_prior)

        mmd_z = (k_z - k_z.diag().diag()).sum() / ((N - 1) * N)
        mmd_z_prior = (k_z_prior - k_z_prior.diag().diag()).sum() / ((N - 1) * N)
        mmd_cross = k_cross.sum() / (N**2)

        mmd_loss = mmd_z + mmd_z_prior - 2 * mmd_cross

        return (
            recon_loss.mean(dim=0) + self.reg_weight * mmd_loss,
            (recon_loss).mean(dim=0),
            mmd_loss,
        )
    def imq_kernel(self, z1, z2):
        """Returns a matrix of shape [batch x batch] containing the pairwise kernel computation"""
        z1 = z1.view(z1.shape[0], -1)
        z2 = z2.view(z2.shape[0], -1)

        Cbase = (
            2.0 * self.latent_dim * self.kernel_bandwidth**2
        )

        k = 0

        for scale in self.scales:
            C = scale * Cbase
            k += C / (C + torch.norm(z1.unsqueeze(1) - z2.unsqueeze(0), dim=-1) ** 2)

        return k

    def rbf_kernel(self, z1, z2):
        """Returns a matrix of shape [batch x batch] containing the pairwise kernel computation"""
        z1 = z1.view(z1.shape[0], -1)
        z2 = z2.view(z2.shape[0], -1)

        C = 2.0 * self.latent_dim * self.kernel_bandwidth**2

        k = torch.exp(-torch.norm(z1.unsqueeze(1) - z2.unsqueeze(0), dim=-1) ** 2 / C)

        return k

In [ ]:
from torchvision import models
from torchsummary import summary
import os

AE_net = WassersteinAE(channel_in=3, ch=64, blocks=(1, 2, 4, 8), latent_channels=64).to(device)
optimizer = optim.Adam(AE_net.parameters(), lr=lr)
#scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = scheduler_gamma)
summary(AE_net, (3, 128, 128))
loss_log = []


In [ ]:
nepoch = 250


for epoch in trange(start_epoch, nepoch, leave=False):
    train_loss = 0
    train_recon_loss = 0
    train_kld_loss = 0

    AE_net.train()
    for i, images in enumerate(tqdm(train_loader, leave=False)):
        images = images.to(device)

        recon_img = AE_net(images)['recon_x']
        #VAE loss
        kl_loss_ = AE_net(images)['mmd_loss']
        mse_loss = AE_net(images)['recon_loss']

        loss = AE_net(images)['loss']

        train_loss += loss.item()
        train_recon_loss += mse_loss.item()
        train_kld_loss += kl_loss_.item()
        #train_perceptual_loss += feature_loss.item()

        loss_log.append(loss.item())
        AE_net.zero_grad()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(AE_net.parameters(), max_norm=1.0)

        optimizer.step()

    avg_loss = train_loss / len(train_loader)  # Calculate average loss for the epoch
    avg_recon_loss = train_recon_loss / len(train_loader)
    avg_kld_loss = train_kld_loss / len(train_loader)
    #avg_perceptual_loss = train_perceptual_loss / len(train_loader)

    #In eval mode the model will use mu as the encoding instead of sampling from the distribution
    AE_net.eval()
    with torch.no_grad():
        recon_img = AE_net(test_images.to(device))['recon_x']
        img_cat = torch.cat((recon_img.cpu(), test_images), 2)

        vutils.save_image(img_cat,
                          "%s/%s/%s_%d.png" % (save_dir, "Results" , model_name, image_size),
                          normalize=True)

        #Save a checkpoint
        torch.save({
                    'epoch'                         : epoch,
                    'loss_log'                      : loss_log,
                    'model_state_dict'              : AE_net.state_dict(),
                    'optimizer_state_dict'          : optimizer.state_dict()

                     }, save_dir + "/Models/" + model_name + "_" + str(image_size) + ".pt")
    print(f'Epoch {epoch}/{nepoch} - Avg Total Loss: {avg_loss} - Avg Recon Loss: {avg_recon_loss}\
    - Avg KLD Loss: {avg_kld_loss}')


In [ ]:

#Create the save directory if it does note exist
if not os.path.isdir(save_dir + "/Models"):
    os.makedirs(save_dir + "/Models")
if not os.path.isdir(save_dir + "/Results"):
    os.makedirs(save_dir + "/Results")

if load_checkpoint:
    checkpoint = torch.load(save_dir + "/Models/" + model_name + "_" + str(image_size) + ".pt", map_location = "cpu")
    print("Checkpoint loaded")
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    AE_net.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint["epoch"]
    loss_log = checkpoint["loss_log"]
else:
    #If checkpoint does exist raise an error to prevent accidental overwriting
    if os.path.isfile(save_dir + "/Models/" + model_name + "_" + str(image_size) + ".pt"):
        raise ValueError("Warning Checkpoint exists")
    else:
        print("Starting from scratch")


In [ ]:
AE_net.eval()

black_image = torch.zeros((3,128,128))
black_image = TF.to_pil_image(black_image)


#black_image = black_image.unsqueeze(0)
#black_image = torch.FloatTensor(1,3,128,128).zero_().cuda()
black_image = test_transform(black_image)
black_image = black_image.unsqueeze(0).cuda()
black_z = AE_net.encoder(black_image)
bl = AE_net.decoder(black_z)
#black_z,_,_ = vae_net.encoder(bl.cuda())
#bl = vae_net.decoder(black_z)
out = vutils.make_grid(bl, normalize=False)
plt.imshow(out.cpu().detach().numpy().transpose((1, 2, 0)))


In [ ]:
white_image = torch.ones((3,128,128))
white_image = TF.to_pil_image(white_image)

#black_image = black_image.unsqueeze(0)
#black_image = torch.FloatTensor(1,3,128,128).zero_().cuda()
white_image = test_transform(white_image)
white_image = white_image.unsqueeze(0).cuda()
white_z = AE_net.encoder(white_image.cuda())
wl = AE_net.decoder(white_z)
#black_z,_,_ = vae_net.encoder(black_image.cuda())
#bl = vae_net.decoder(black_z)


out = vutils.make_grid(wl, normalize=True)
plt.imshow(out.cpu().detach().numpy().transpose((1, 2, 0)))